# Solomon's notebook - Turnover
## To visualize percentage share of most traded stock in HKEX for the last 5 trading days.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px

from lib import hkexquotations as hkex
from lib import holidays
from lib.hkexquotations import NoQuotationsException

today = datetime.now()
yesterday = today - timedelta(days=1)
end_day = today

try:
    hkex.get_price(today)
except NoQuotationsException:
    end_day = yesterday
    
yesterday = datetime.now() - timedelta(days=1)
bdates = pd.bdate_range(end=end_day, periods=5, holidays=holidays.hk_holidays, freq='C')

topN = 20

In [ ]:
df = hkex.get_price(bdates[4])
df = df.nlargest(topN, ['turnover'])
pie = px.pie(df, names='code', values='turnover', title=f"Turnover Share of the top {topN} as of date {bdates[4]:%Y-%m-%d}")
pie.update_traces(textposition='inside', textinfo='percent+label')

pie.show()

In [ ]:
df = hkex.get_price(bdates[3])
df = df.nlargest(topN, ['turnover'])
pie = px.pie(df, names='code', values='turnover', title=f"Turnover Share of the top {topN} as of date {bdates[3]:%Y-%m-%d}")
pie.update_traces(textposition='inside', textinfo='percent+label')

pie.show()

In [ ]:
df = hkex.get_price(bdates[2])
df = df.nlargest(topN, ['turnover'])
pie = px.pie(df, names='code', values='turnover', title=f"Turnover Share of the top {topN} as of date {bdates[2]:%Y-%m-%d}")
pie.update_traces(textposition='inside', textinfo='percent+label')

pie.show()

In [ ]:
df = hkex.get_price(bdates[1])
df = df.nlargest(topN, ['turnover'])
pie = px.pie(df, names='code', values='turnover', title=f"Turnover Share of the top {topN} as of date {bdates[1]:%Y-%m-%d}")
pie.update_traces(textposition='inside', textinfo='percent+label')

pie.show()

In [ ]:
df = hkex.get_price(bdates[0])
df = df.nlargest(topN, ['turnover'])
pie = px.pie(df, names='code', values='turnover', title=f"Turnover Share of the top {topN} as of date {bdates[0]:%Y-%m-%d}")
pie.update_traces(textposition='inside', textinfo='percent+label')

pie.show()